In [ ]:

from src.render.loops.progress import PreviewConfig, ProgressDisplay
from src.render.render_config import RenderConfig
from src.render import LinearRayCaster
from src.geometry.primitives.box import Box
from src.geometry.primitives.torus import Torus
from src.geometry.primitives.cylinder import Cylinder
from src.scene.primitive import Object
from src.material.textures.perlin_noise import PerlinNoise
from src.material.material.phong_material import PhongMaterial
from src import Color, Camera, Resolution, Vertex, PointLight, Vector, Scene, Square, Sphere, Triangle, AmbientLight

WHITE = Color.custom_rgb(255, 255, 255)

def M(name, base, *, shin=30, ior=1.5, refl=0.0, transp=0.0, nscale=None, nstr=0.0):
    return PhongMaterial(
        name=name,
        base_color=base,
        spec_color=WHITE,
        shininess=shin,
        ior=ior,
        reflectivity=refl,
        transparency=transp,
        normal_noise=None if nscale is None else PerlinNoise(scale=nscale, strength=nstr),
    )

def default_camera():
    return Camera(
        fov=50,
        aspect_ratio=Resolution.custom(width=1050, height=500).aspect_ratio,
        origin=Vertex(0, 0, 0),
        direction=Vector(0, 0, -1),
    )

def default_lights():
    return [
        PointLight(position=Vertex(0, 3, 0),   intensity=800.0, falloff=0.0015),
        AmbientLight(intensity=0.2),
    ]


def make_scene_mixed_demo() -> Scene:
    glossy_red = M("glossy_red", Color.custom_rgb(255, 10, 40),  shin=30,  refl=0.2, nscale=2,  nstr=0.1)
    glass_blue = M("glass_blue", Color.custom_rgb(10, 100, 255), shin=50,  refl=0.1, transp=0.9, nscale=4, nstr=0.1)
    sand       = M("sand",       Color.custom_rgb(210, 178, 128),shin=10,  ior=1.3, nscale=8,  nstr=0.08)
    golden     = M("golden",     Color.custom_rgb(255, 215, 0),  shin=100, refl=0.8, nscale=1.5, nstr=0.05)
    diamond    = M("diamond",    Color.custom_rgb(185, 242, 255),shin=300, ior=2.42, refl=0.1, transp=0.95, nscale=5, nstr=0.01)
    mirror     = M("mirror",     Color.custom_rgb(200, 200, 200),shin=500, ior=1.0, refl=1.0)

    scene = Scene(
        camera=default_camera(),
        objects=[
            Object(Square(Vertex(-7, -1, -15), Vertex(7, -1, 10)), sand),
            Object(Sphere(Vertex(-0.5, 0.2, -6), 1.0), diamond),
            Object(Triangle(Vertex(2, 0, -8), Vertex(3, 1.5, -7), Vertex(4, 0, -8)), mirror),
            Object(Cylinder(Vertex(-0.5, -0.6, -2.0), Vertex(-0.9, 0.6, -2.2), 0.4), glass_blue),
            Object(Torus(Vertex(1, -0.85, -2.85), radius_major=0.6, radius_tube=0.15), M("bumpy_blue", Color.custom_rgb(30, 144, 255), shin=50, refl=0.3, nscale=3, nstr=0.2)),
            Object(Box(Vertex(-6, -0.7, -10), Vertex(-3, 1, -8)), glossy_red),
            Object(Triangle(Vertex(7, 1, -11), Vertex(3, 4, -11), Vertex(1, -0.7, -9)), golden),
        ],
        lights=default_lights(),
    )
    scene.validate()
    return scene


def make_scene_geometry_gallery() -> Scene:
    matte = M("matte", Color.custom_rgb(200, 200, 200), shin=5)
    floor = M("floor", Color.custom_rgb(180, 180, 180), shin=5)

    scene = Scene(
        camera=default_camera(),
        objects=[
            Object(Square(Vertex(-8, -1, -20), Vertex(8, -1, 5)), floor),
            Object(Sphere(Vertex(-3, 0, -8), 1.0), matte),
            Object(Cylinder(Vertex(0, -1, -8), Vertex(0, 1, -8), 0.7), matte),
            Object(Torus(Vertex(3, 0, -8), 0.9, 0.25), matte),
        ],
        lights=default_lights(),
    )
    scene.validate()
    return scene


def make_scene_noise_demo() -> Scene:
    bumpy = M("bumpy", Color.custom_rgb(120, 160, 255), shin=30, nscale=6, nstr=0.2)
    floor = M("floor", Color.custom_rgb(200, 200, 200), shin=5, nscale=12, nstr=0.1)

    scene = Scene(
        camera=default_camera(),
        objects=[
            Object(Square(Vertex(-8, -1, -20), Vertex(8, -1, 5)), floor),
            Object(Sphere(Vertex(0, 0, -7), 1.2), bumpy),
        ],
        lights=default_lights(),
    )
    scene.validate()
    return scene

def transparency_test() -> Scene:
    transp_0  = M("transp_0",  Color.custom_rgb(0, 0, 0), shin=64, ior=1.5, refl=0.0, transp=0.0)
    transp_25 = M("transp_25", Color.custom_rgb(255, 0, 0), shin=64, ior=1.5, refl=0.0, transp=0.25)
    transp_50 = M("transp_50", Color.custom_rgb(0, 255, 0), shin=64, ior=1.5, refl=0.0, transp=0.50)
    transp_75 = M("transp_75", Color.custom_rgb(0, 0, 255), shin=64, ior=1.5, refl=0.0, transp=0.75)
    transp_90 = M("transp_90", Color.custom_rgb(255, 255, 0), shin=64, ior=1.5, refl=0.0, transp=0.90)
    floor = M("floor", Color.custom_rgb(10, 169, 224), shin=5)

    scene = Scene(
        camera=default_camera(),
        objects=[
            Object(Square(Vertex(-8, -1, -20), Vertex(8, -1, 5)), floor),
            Object(Sphere(Vertex(-4, 0, -8), 1.0), transp_0),
            Object(Sphere(Vertex(-2, 0, -8), 1.0), transp_25),
            Object(Sphere(Vertex(0, 0, -8), 1.0), transp_50),
            Object(Sphere(Vertex(2, 0, -8), 1.0), transp_75),
            Object(Sphere(Vertex(4, 0, -8), 1.0), transp_90),
        ],
        lights=default_lights(),
    )
    scene.validate()
    return scene

def shiness_test_5_balls() -> Scene:
    #form 0 to 32
    shin_0  = M("shin_0",  Color.custom_rgb(0, 0, 0), shin=0,   ior=1.5, refl=0.0, transp=0.0)
    shin_8  = M("shin_25", Color.custom_rgb(0, 0, 0), shin=8,   ior=1.5, refl=0.0, transp=0.0)
    shin_16 = M("shin_50", Color.custom_rgb(0, 0, 0), shin=16,  ior=1.5, refl=0.0, transp=0.0)
    shin_24 = M("shin_75", Color.custom_rgb(0, 0, 0), shin=24,  ior=1.5, refl=0.0, transp=0.0)
    shin_32 = M("shin_100",Color.custom_rgb(0, 0, 0),shin=32,  ior=1.5, refl=0.0, transp=0.0)

    floor = M("floor", Color.custom_rgb(10, 169, 224), shin=5)
    scene = Scene(
        camera=default_camera(),
        primitives=[
            Object(Square(Vertex(-8, -1, -20), Vertex(8, -1, 5)), floor),
            Object(Sphere(Vertex(-4, 0, -8), 1.0), shin_0),
            Object(Sphere(Vertex(-2, 0, -8), 1.0), shin_8),
            Object(Sphere(Vertex(0, 0, -8), 1.0), shin_16),
            Object(Sphere(Vertex(2, 0, -8), 1.0), shin_24),
            Object(Sphere(Vertex(4, 0, -8), 1.0), shin_32),
        ],
        lights=default_lights(),
    )
    scene.validate()
    return scene

def reflectivity_test() -> Scene:
    refl_0  = M("refl_0",  Color.custom_rgb(0, 0, 0), shin=64, ior=1.5, refl=0.0, transp=0.0)
    refl_25 = M("refl_25", Color.custom_rgb(0, 0, 0), shin=64, ior=1.5, refl=0.25, transp=0.0)
    refl_50 = M("refl_50", Color.custom_rgb(0, 0, 0), shin=64, ior=1.5, refl=0.50, transp=0.0)
    refl_75 = M("refl_75", Color.custom_rgb(0, 0, 0), shin=64, ior=1.5, refl=0.75, transp=0.0)
    refl_100 = M("refl_100", Color.custom_rgb(0, 0, 0), shin=64, ior=1.5, refl=0.99, transp=0.0)
    floor = M("floor", Color.custom_rgb(10, 169, 224), shin=5)

    scene = Scene(
        camera=default_camera(),
        primitives=[
            Object(Square(Vertex(-8, -1, -20), Vertex(8, -1, 5)), floor),
            Object(Sphere(Vertex(-4, 0, -8), 1.0), refl_0),
            Object(Sphere(Vertex(-2, 0, -8), 1.0), refl_25),
            Object(Sphere(Vertex(0, 0, -8), 1.0), refl_50),
            Object(Sphere(Vertex(2, 0, -8), 1.0), refl_75),
            Object(Sphere(Vertex(4, 0, -8), 1.0), refl_100),
        ],
        lights=default_lights(),
    )
    scene.validate()
    return scene

def ior_test_5_balls() -> Scene:
    ior_1_0 = M("ior_1_0", Color.custom_rgb(255, 0, 0), shin=124, ior=1.0, refl=0.0, transp=0.9)
    ior_1_3 = M("ior_1_3", Color.custom_rgb(0, 255, 0), shin=124, ior=1.3, refl=0.0, transp=0.9)
    ior_1_5 = M("ior_1_5", Color.custom_rgb(0, 0, 255), shin=124, ior=1.5, refl=0.0, transp=0.9)
    ior_1_7 = M("ior_1_7", Color.custom_rgb(255, 255, 0), shin=124, ior=1.7, refl=0.0, transp=0.9)
    ior_2_0 = M("ior_2_0", Color.custom_rgb(0, 255, 255), shin=124, ior=2.0, refl=0.0, transp=0.9)
    floor = M("floor", Color.custom_rgb(10, 169, 224), shin=5)

    scene = Scene(
        camera=default_camera(),
        primitives=[
            Object(Square(Vertex(-8, -1, -20), Vertex(8, -1, 5)), floor),
            Object(Sphere(Vertex(-4, 0, -8), 1.0), ior_1_0),
            Object(Sphere(Vertex(-2, 0, -8), 1.0), ior_1_3),
            Object(Sphere(Vertex(0, 0, -8), 1.0), ior_1_5),
            Object(Sphere(Vertex(2, 0, -8), 1.0), ior_1_7),
            Object(Sphere(Vertex(4, 0, -8), 1.0), ior_2_0),
        ],
        lights=default_lights(),
    )
    scene.validate()
    return scene

SCENES = {
    "test_ior_5_balls": ior_test_5_balls,
    "test_reflectivity_5_balls": reflectivity_test,
    "test_shiness_5_balls": shiness_test_5_balls,
    "5_balls": transparency_test,
    "mixed": make_scene_mixed_demo,
    "geo": make_scene_geometry_gallery,
    "noise": make_scene_noise_demo,
}

scene = SCENES["test_shiness_5_balls"]()

my_ray_tracer = LinearRayCaster(
    scene=scene,
    render_config=RenderConfig(resolution=Resolution.custom(width=1050, height=500), samples_per_pixel=15, max_depth=15),
    preview_config=PreviewConfig(progress_display=ProgressDisplay.TQDM_IMAGE_PREVIEW),
)

my_ray_tracer.render("images/out.png")

